In [ ]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 41.1 MB/s eta 0:00:00


# Import Lib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from transformers import get_linear_schedule_with_warmup
import re, os, time, gc, codecs, string, math, heapq
from typing import List
from pyvi import ViTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import corpus_bleu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SRC_LANGUAGE = 'eng'
TGT_LANGUAGE = 'vi'

ModuleNotFoundError: ignored

In [ ]:
NUM_EPOCHS = 50
SAMPLES = 1e9
MAX_LEN = 50
continue_training_from_checkpoint = 0 # 0: retrain
checkpoint_path = f'/kaggle/input/translate-machine-model-state-dict/checkpoint_{continue_training_from_checkpoint}.pth'
batch_size = 15
d_model = 512
num_heads = 8
d_ff = 512
batch_size = 64
num_encoder_layers = 6
num_decoder_layers = 6
dropout = 0.1

# Data Preparing

In [ ]:
!git clone https://github.com/KCDichDaNgu/KC4.0_MultilingualNMT.git
DATA_DIR = '/kaggle/working/KC4.0_MultilingualNMT/data/iwslt_en_vi'

Cloning into 'KC4.0_MultilingualNMT'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 268 (delta 5), reused 3 (delta 3), pack-reused 255
Receiving objects: 100% (268/268), 45.75 MiB | 19.37 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
def preprocess_sentence(text, language):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Handle special characters
#     text = re.sub(r'\d+', 'num', text) # Replace all digits with 'num'
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespaces with a single space
        
#     Tokenize the text
#     words = word_tokenize(text)
    
#     if language == 'eng' and len(words) > 50:
#         stop_words = set(stopwords.words('english'))
#         words = [w for w in words if not w in stop_words]
#     if language == 'vi' and len(words) > 50:
#         f = codecs.open('/kaggle/input/sentiment-analysis-foody/vietnamese-stopwords.txt', encoding='utf-8')
#         stop_words = []
#         for i, line in enumerate(f):
#             line = repr(line)
#             line = line[1:len(line)-3]
#             stop_words.append(line)
#         words = [w for w in words if not w in stop_words]
    # Deal with rare or infrequent words
    return text.strip()

def load_data(source_file, target_file, number_of_examples, MAX_LEN):
    source_sents = open(source_file, "r").readlines()
    target_sents = open(target_file, "r").readlines()
    assert len(source_sents) == len(target_sents)

    source_data, target_data = [], []

    for src_sentence, trg_sentence in zip(source_sents, target_sents):
        if(len(source_data) >= number_of_examples):
            break
        if(len(src_sentence.split()) > MAX_LEN or len(trg_sentence.split())> MAX_LEN):
            continue
        source_data.append(preprocess_sentence(src_sentence, SRC_LANGUAGE))
        target_data.append(preprocess_sentence(trg_sentence, TGT_LANGUAGE))
    return source_data, target_data

source_data, target_data = load_data(DATA_DIR+"/train.en", DATA_DIR+"/train.vi", SAMPLES, MAX_LEN)
src_tokenized = [word_tokenize(i) for i in source_data]
tgt_tokenized = [ViTokenizer.tokenize(i).split() for i in target_data] 

In [ ]:
tgt_tokenized[0]

['khoa_học', 'đằng', 'sau', 'một', 'tiêu_đề', 'về', 'khí_hậu']

In [ ]:
vocab_transform = {}
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
def yield_tokens(lang_tokenized):
    for i in lang_tokenized:
        yield i
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(src_tokenized), min_freq=1, specials=special_symbols, special_first=True)
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(tgt_tokenized), min_freq=1, specials=special_symbols, special_first=True)
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)
src_vocab_size = len(vocab_transform[SRC_LANGUAGE])
tgt_vocab_size = len(vocab_transform[TGT_LANGUAGE])

In [ ]:
print(vocab_transform[SRC_LANGUAGE]['The'])
print(vocab_transform[TGT_LANGUAGE]['4'])

0
394


In [ ]:
print(vocab_transform[SRC_LANGUAGE](['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']))
print(vocab_transform[TGT_LANGUAGE](tgt_tokenized[0]))

[0, 1255, 2376, 5493, 7942, 123, 4, 6805, 1315]
[209, 898, 82, 7, 2925, 30, 970]


# Create Data for Training

In [ ]:
def sequential_transform(*transforms):
    def func(tokens_input):
        for transform in transforms:
            tokens_input = transform(tokens_input)
        return tokens_input
    return func

def tensor_transform(tokens_ids: List[int]):
    # Adding bos and eos
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(tokens_ids), torch.tensor([EOS_IDX])))


text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transform(vocab_transform[ln], tensor_transform)

In [ ]:
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    # Word to index
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample))

    # Padding 
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    
    # size: [max_lenght, batch_size]
    return src_batch.to(torch.int64), tgt_batch.to(torch.int64)

train_iter = [i for i in zip(src_tokenized, tgt_tokenized)]
train_dataloader = DataLoader(train_iter, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Modeling

## Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## Token Embedding

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(TokenEmbedding, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
    
    def forward(self, tokens):
        # math.sqrt(self.d_model) for stability training
        return self.token_embedding(tokens.long()) * math.sqrt(self.d_model)


## Attention

In [ ]:
# Self Attention
def attention(q, k, v, mask=None, dropout=None):
    d_k = q.size(-1)
    dot_product = torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d_k)
    if mask is not None:
        for i in range(0, dot_product.size(0), mask.size(0)):
            dot_product[i: i+mask.size(0)] = (dot_product[i: i+mask.size(0)]).masked_fill(mask == 0, -1e9)
        scores = dot_product
    else:
        scores = dot_product
        
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.bmm(p_attn, v)
# MultiHeadAttention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.linears = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(4)])
        self.dropout = nn.Dropout(dropout)
        self.d_k = d_model // num_heads

    def forward(self, q, k, v, mask=None):

        q = self.linears[0](q)
        k = self.linears[1](k)
        v = self.linears[2](v)
        def _split_heads(tensor):
            bsz, length, embed_dim = tensor.size()
            tensor = tensor.reshape(bsz, length, self.num_heads, self.d_k).transpose(1, 2).reshape(bsz * self.num_heads, -1, self.d_k)
            return tensor
        q = _split_heads(q)
        k = _split_heads(k)
        v = _split_heads(v)

        output = attention(q, k, v, mask=mask,  dropout=self.dropout)

        bsz_heads, length, d_k = output.size()
        bsz = bsz_heads // num_heads
        output = output.reshape(bsz, num_heads, length, self.d_k).transpose(1, 2).reshape(bsz, length, -1)

        return self.linears[3](output)


## LayerNorm

In [ ]:
# Calculate LayerNorm before Skip-Connection, avoid vanishing gradient
# https://arxiv.org/abs/1910.05895
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))


## Position-wise Feed-Forward

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.d_model = d_model 
        self.d_ff = d_ff
        self.dropout = nn.Dropout(dropout)
        self.linear_in = nn.Linear(d_model, d_ff)
        self.linear_out = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        y = F.relu(self.linear_in(x))
        y = self.linear_out(self.dropout(y))
        return y


## Encoder

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.SelfMultiHeadAttention = MultiHeadAttention(d_model, num_heads, dropout)
        self.FeedForward = FeedForward(d_model, d_ff, dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(2)])
        self.d_model = d_model

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.SelfMultiHeadAttention(x, x, x, mask))
        return self.sublayer[1](x, self.FeedForward)

# stack encoder layers
class Encoder(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, num_layers, dropout=0.1):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, d_ff, num_heads, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)



## Decoder


In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.SelfMultiHeadAttention = MultiHeadAttention(d_model, num_heads, dropout)
        self.MultiHeadAttention = MultiHeadAttention(d_model, num_heads, dropout)
        self.FeedForward = FeedForward(d_model, d_ff, dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(3)])
        self.d_model = d_model
    
    def forward(self, x, memory, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.SelfMultiHeadAttention(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.MultiHeadAttention(x, memory, memory))
        return self.sublayer[2](x, self.FeedForward)

# stack decoder layers
class Decoder(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, num_layers, dropout=0.1):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, d_ff, num_heads, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
    
    def forward(self, x, memory, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, tgt_mask)
        return self.norm(x)


## Transformer Model

In [ ]:
class Transformer(nn.Module):
    def __init__(self,
                 num_encoder_layers,
                 num_decoder_layers,
                 d_model, d_ff,
                 src_vocab_size,
                 tgt_vocab_size,
                 num_heads, dropout):
        super(Transformer, self).__init__()
        self.pos_encoding = PositionalEncoding(d_model, dropout)
        self.src_tok_emb = TokenEmbedding(d_model, src_vocab_size)
        self.tgt_tok_emb = TokenEmbedding(d_model, tgt_vocab_size)
        self.encoder = Encoder(d_model, d_ff, num_heads, num_encoder_layers, dropout)
        self.decoder = Decoder(d_model, d_ff, num_heads, num_decoder_layers, dropout)
        self.generator = nn.Linear(d_model, tgt_vocab_size)
    
    def forward(self, src_input_tensor, tgt_input_tensor, src_mask, tgt_mask, padding_mask=None):
        src_emb = self.pos_encoding(self.src_tok_emb(src_input_tensor))
        tgt_emb = self.pos_encoding(self.tgt_tok_emb(tgt_input_tensor))
        output_encoder = self.encoder(src_emb, src_mask)
        output_decoder = self.decoder(tgt_emb, output_encoder, tgt_mask)
        return self.generator(output_decoder)
    
    def encode(self, src_input_tensor, src_mask):
        src_emb = self.pos_encoding(self.src_tok_emb(src_input_tensor))
        output_encoder = self.encoder(src_emb, src_mask)
        return output_encoder
    def decode(self, tgt_input_tensor, tgt_mask, memory):
        tgt_emb = self.pos_encoding(self.tgt_tok_emb(tgt_input_tensor))
        output_decoder = self.decoder(tgt_emb, memory, tgt_mask)
        return self.generator(output_decoder)

# Training Model


### Utils

In [ ]:
def create_mask(batch, lang): 
    assert lang == 'src' or lang == 'tgt'
    bsz, length = batch.size()
    mask = torch.ones(bsz, length, length)   
    # For source sequences, return a mask with all ones
    if lang == 'src':
        return mask.to(device)
    # For target sequences, return a mask that only attends to positions before the current position
    else:
        return torch.triu(mask).transpose(1, 2).to(device)

def create_padding_mask(batch):
    bsz, length = batch.size()
    # The mask is 0 for padding tokens and 1 for all other tokens.
    padding_mask = (batch != PAD_IDX).unsqueeze(-1)
    return padding_mask.to(device)

def create_std_mask(padding_mask, mask, position):
    assert position == 'encoder' or position == 'decoderI' or position == 'decoderII'
    # "encoder": Encoder don't attend to padding token in src sentence
    # "decoderI": Decoder don't attend to padding token in tgt sentence and only focus on positions before the current position
    # "decoderII": Decoder don't attend to padding token in src sentence 
    if position == 'encoder' or position == 'decoderI':
        padding_mask = torch.bmm(padding_mask.float(), padding_mask.transpose(1, 2).float())
        mask = padding_mask * mask
    else:
        mask = torch.bmm(mask.float(), padding_mask.transpose(1, 2).float())
    return mask.long().to(device)


# Let's go bruhhh

In [ ]:
def train_epoch(epoch, model, optimizer, scheduler):
    model.train()
    losses = 0
    total = 0
    for batch_id, (src, tgt) in enumerate(train_dataloader):
        src = src.transpose(0, 1).to(device)
        tgt = tgt.transpose(0, 1).to(device)

        tgt_input = tgt[:, :-1]
        src_mask, tgt_mask = create_mask(src, 'src'), create_mask(tgt_input, 'tgt')
        src_padding_mask, tgt_padding_mask = create_padding_mask(src), create_padding_mask(tgt_input)
        src_mask = create_std_mask(src_padding_mask, src_mask, 'encoder').long()
        tgt_mask = create_std_mask(tgt_padding_mask, tgt_mask, 'decoderI').long()
        last_mask = create_std_mask(src_padding_mask, tgt_padding_mask, 'decoderII').long()
        
        logits = model(src, tgt_input, src_mask , tgt_mask, last_mask)
        optimizer.zero_grad()

        tgt_out = tgt[:, 1:]
        
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
            

        optimizer.step()
        scheduler.step()
        losses += loss.item()
        total += tgt.size(0)
        del tgt_input
        del tgt_mask
        del src_mask
        del last_mask
        del src_padding_mask
        del tgt_padding_mask
        del loss
        gc.collect()
        torch.cuda.empty_cache()
        if batch_id % 100 == 0 or batch_id < 10:
            print(f"""Total loss: {losses:.4f} | Total: {total} | Loss per batch: {losses/(batch_id + 1):.4f}""")
    
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': losses / len(train_dataloader),
    }
    if epoch > continue_training_from_checkpoint and epoch % 10 == 0:
        torch.save(checkpoint, f'/kaggle/working/checkpoint_{epoch}.pth')
    del checkpoint
    gc.collect()
    return losses / len(train_dataloader)

In [ ]:
# Optimize ram usage 
del source_data
del target_data
del src_tokenized
del tgt_tokenized
del train_iter
gc.collect()
torch.cuda.empty_cache()

transformer = Transformer(num_encoder_layers, num_decoder_layers, d_model, d_ff, src_vocab_size, tgt_vocab_size, num_heads, dropout)
transformer.to(device)
optimizer = torch.optim.Adam(transformer.parameters(), lr=2e-5, betas=(0.9, 0.98), eps=1e-9)
num_training_steps = len(train_dataloader) * NUM_EPOCHS
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

if continue_training_from_checkpoint == 0:
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
else:
    checkpoint = torch.load(checkpoint_path)
    transformer.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict']) 
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Epoch: {epoch}, Avg train loss: {loss:.3f}")
    
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(continue_training_from_checkpoint+1, NUM_EPOCHS+1):
    print(f'======== Epoch {epoch} / {NUM_EPOCHS} ========')
    print('Training...')
    start_time = time.time()
    train_loss = train_epoch(epoch, transformer, optimizer, scheduler)
    end_time = time.time()
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 50, Avg train loss: 3.077


# Beam search

In [ ]:
def beam_search_decode(model, src, src_mask, max_len, beam_size):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)

    # Initialize the beam with the start-of-sequence token
    beam = [(0.0, [BOS_IDX])]

    # Iterate until the maximum length or the end-of-sequence token is generated
    for i in range(max_len-1):
        new_beam = []
#         print(f'Step: {i}')
#         print('----------------')
        # Generate candidates for each beam
        for score, sequence in beam:
            # Get the last generated token
            last_token = sequence[-1]
#             print(last_token)

            # Stop generating if the end-of-sequence token is generated
            if last_token == EOS_IDX:
                new_beam.append((score, sequence))
                continue

            # Decode the current sequence
            ys = torch.tensor(sequence).unsqueeze(0).to(device)
            tgt_mask = create_mask(ys, 'tgt').to(device)
            out = model.decode(ys, tgt_mask, memory)
            prob = out[:, -1]

            # Get the top k candidates for the next token
            topk_scores, topk_indices = torch.topk(prob, k=beam_size)

            # Add the top k candidates to the beam
            for j in range(beam_size):
                new_score = score + topk_scores[0, j].item()
                new_sequence = sequence + [topk_indices[0, j].item()]
                new_beam.append((new_score, new_sequence))

        # Keep only the top k candidates in the beam
        beam = heapq.nlargest(beam_size, new_beam, key=lambda x: x[0])
#         print('------------')

    # Return the sequence with the highest score in the beam
    return beam[0][1]


In [ ]:
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src_token = word_tokenize(preprocess_sentence(src_sentence, SRC_LANGUAGE))
    src = torch.tensor(vocab_transform[SRC_LANGUAGE](src_token)).unsqueeze(0)
    num_tokens = src.size(-1)
    print(src.size())
    tgt_tokens = beam_search_decode(model, src, src_mask=create_mask(src, 'src'), max_len=num_tokens +5, beam_size=10)
    return (" ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(tgt_tokens))).replace("<bos>", "<bos>").replace("<eos>", "<eos>").strip()

In [ ]:
print(translate(transformer, 'science haha'))

torch.Size([1, 2])
<bos> ngành khoa_học ngừng khoa_học sử_dụng khoa_học


# Evaluation

In [ ]:
del train_dataloader
gc.collect()

72

In [ ]:
def calculate_bleu_score(model, data_loader):
    # Set the model to evaluation mode
    print("Bleu Score")
    print("Calculating...")
    model.eval()

    # Initialize the references and hypotheses lists
    references = []
    hypotheses = []

    # Iterate over the validation set and generate translations
    with torch.no_grad():
        for batch_id, (src, tgt) in enumerate(data_loader):
            if batch_id % 100 == 0:
                print(f"Processed {batch_id}/{len(data_loader)}")
            src = src.transpose(0,1).to(device)
            tgt = tgt.transpose(0,1).to(device)
            # Generate a translation using beam search decoding
            translation = beam_search_decode(model, src, src_mask=create_mask(src, 'src'), max_len=tgt.size(1) + 5, beam_size=10)

            # Convert the tokens to strings and append to the lists
#             print([tgt.cpu().numpy().tolist()])
            references.extend([tgt.cpu().numpy().tolist()])
#             print(2)
            hypotheses.append(translation)

    # Calculate the BLEU score
#     print((references))
#     print((hypotheses))
    bleu_score = corpus_bleu(references, hypotheses)

    return bleu_score

In [ ]:
# Preparing Validation dataset
source_data, target_data = load_data(DATA_DIR+"/tst2012.en", DATA_DIR+"/tst2012.vi", 1e9, 1e9)
src_tokenized = [word_tokenize(i) for i in source_data]
tgt_tokenized = [ViTokenizer.tokenize(i).split() for i in target_data]
val_iter = [i for i in zip(src_tokenized, tgt_tokenized)]
val_dataloader = DataLoader(val_iter, batch_size=1, shuffle=True, collate_fn=collate_fn)
# Evaluation
calculate_bleu_score(transformer, val_dataloader)

Bleu Score
Calculating...
Processed 0/1553
Processed 100/1553
Processed 200/1553
Processed 300/1553
Processed 400/1553
Processed 500/1553
Processed 600/1553
Processed 700/1553
Processed 800/1553
Processed 900/1553
Processed 1000/1553
Processed 1100/1553
Processed 1200/1553
Processed 1300/1553
Processed 1400/1553
Processed 1500/1553


0.07085401329851057

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Define reference and candidate translations
references = [['the', 'cat', 'is', 'on', 'the', 'mat'],
              ['there', 'is', 'a', 'cat', 'on', 'the', 'mat']]
candidate = ['the', 'cat', 'is', 'on', 'the', 'mat']

# Compute BLEU score between candidate and reference translations
bleu_score = corpus_bleu([references], [candidate])

# Print BLEU score
print(bleu_score)
